In [1]:
import pandas as pd
import numpy as np
from ipynb.fs.full.interp_data import interpolate_scenario_data

Extract data from AR6 database
- "Capacity|Electricity" GW
- "Final Energy|Electricity" EJ/yr
- Policy_category - P1c: NDC

In [2]:
# load data
ar6_world = pd.read_csv("../../data/AR6_data/1668008174375-AR6_Scenarios_Database_R10_regions_v1.1.csv/AR6_Scenarios_Database_R10_regions_v1.1.csv")
ar6_scen = pd.read_excel("../../data/AR6_data/AR6_Scenarios_Database_World_v1.1/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx",sheet_name = "meta_Ch3vetted_withclimate")

In [3]:
ar6_scen_NDC = ar6_scen[ar6_scen["Policy_category"]=="P1c"]
ar6_scen_NDC

,Model,Scenario,Category,Category_name,Category_subset,Subset_Ch4,Category_Vetting_historical,IMP_marker,Literature Reference (if applicable),Policy_category,...,P67 peak warming (FaIRv1.6.2),Median warming in 2100 (FaIRv1.6.2),Median year of peak warming (FaIRv1.6.2),Exceedance Probability 1.5C (FaIRv1.6.2),Exceedance Probability 2.0C (FaIRv1.6.2),Exceedance Probability 3.0C (FaIRv1.6.2),IMP_color_rgb,IMP_color_hex,Category_color_rgb,Category_color_hex
9,AIM/CGE 2.1,CD-LINKS_INDCi,C7,C7: limit warming to 4°C (>50%),C7,NDCs announced prior to COP26,C7,non-IMP,https://doi.org/10.1007/s10584-020-02837-9,P1c,...,3.329269,3.071320,2100,1.000000,0.995530,0.552973,NaN,NaN,"241, 136, 114",F18872
27,AIM/CGE 2.2,EN_INDCi2100,C6,C6: limit warming to 3°C (>50%),C6,NDCs announced prior to COP26,C6,non-IMP,https://doi.org/10.1038/s41558-021-01215-2; ht...,P1c,...,2.569700,2.366181,2100,0.996424,0.824765,0.111757,NaN,NaN,"250, 193, 130",FAC182
67,COFFEE 1.1,CO_NDCplus,C6,C6: limit warming to 3°C (>50%),C6,NDCs announced prior to COP26,C6,non-IMP,https://doi.org/10.1038/s41467-021-26595-z,P1c,...,2.839779,2.615368,2100,0.998212,0.940992,0.232007,NaN,NaN,"250, 193, 130",FAC182
95,COFFEE 1.1,EN_INDCi2100,C6,C6: limit warming to 3°C (>50%),C6,NDCs announced prior to COP26,C6,non-IMP,https://doi.org/10.1038/s41558-021-01215-2; ht...,P1c,...,2.949941,2.714000,2100,0.999106,0.961109,0.297273,NaN,NaN,"250, 193, 130",FAC182
96,COFFEE 1.1,EN_INDCi2100_COV_NDCp,C6,C6: limit warming to 3°C (>50%),C6,NaN,C6,non-IMP,https://doi.org/10.1038/s41558-021-01215-2; ht...,P1c,...,2.775590,2.551087,2100,0.997765,0.919982,0.195351,NaN,NaN,"250, 193, 130",FAC182
130,EPPA 6,ParisFlat,C6,C6: limit warming to 3°C (>50%),C6,NDCs announced prior to COP26,C6,non-IMP,https://doi.org/10.1016/j.ijggc.2019.05.016,P1c,...,3.166668,2.910515,2100,1.000000,0.987483,0.436299,NaN,NaN,"250, 193, 130",FAC182
131,EPPA 6,ParisForever,C7,C7: limit warming to 4°C (>50%),C7,NaN,C7,non-IMP,"Reilly, J., R. Prinn, Y.H. Chen, A. Sokolov, X...",P1c,...,3.745787,3.464629,2100,1.000000,0.999106,0.796156,NaN,NaN,"241, 136, 114",F18872
142,GCAM 5.3,NGFS2_Nationally Determined Contributions (NDCs),C5,C5: limit warming to 2.5°C (>50%),C5,NDCs announced prior to COP26,C5,non-IMP,NGFS Climate Scenarios for central banks and s...,P1c,...,2.421362,2.231890,2100,0.992401,0.733125,0.061690,NaN,NaN,"140, 167, 208",8CA7D0
199,GEM-E3_V2021,EN_INDCi2100,C7,C7: limit warming to 4°C (>50%),C7,NDCs announced prior to COP26,C7,non-IMP,https://doi.org/10.1038/s41558-021-01215-2; ht...,P1c,...,3.035188,2.794861,2100,0.999553,0.970943,0.354493,NaN,NaN,"241, 136, 114",F18872
200,GEM-E3_V2021,EN_INDCi2100_COV,C6,C6: limit warming to 3°C (>50%),C6,NDCs announced prior to COP26,C6,non-IMP,https://doi.org/10.1038/s41558-021-01215-2; ht...,P1c,...,2.962134,2.730213,2100,0.999553,0.964238,0.305767,NaN,NaN,"250, 193, 130",FAC182


In [4]:
#var = "Capacity|Electricity"
#value_name = "GW"
#file_name = "AR6_Electricity_GW_R10_capacity.csv"

var_list = ["Secondary Energy|Electricity","Secondary Energy|Electricity|Coal|w/o CCS",
           "Secondary Energy|Electricity|Gas|w/o CCS","Secondary Energy|Electricity|Solar",
            "Secondary Energy|Electricity|Wind"]
value_name_list  = ["EJ/yr","EJ/yr","EJ/yr","EJ/yr","EJ/yr"]
file_name_list = ["AR6_SecEnergy_el_R10_NDC.csv","AR6_coal_el_R10_NDC.csv","AR6_gas_el_R10_NDC.csv",
                 "AR6_solar_el_R10_NDC.csv","AR6_wind_el_R10_NDC.csv"]

for i in np.arange(len(var_list)): 
    var = var_list[i]
    value_name = value_name_list[i]
    file_name = file_name_list[i]
    ar6_world_coal = ar6_world.query("Variable==@var")
    ar6_world_coal = interpolate_scenario_data(ar6_world_coal)
    ar6_world_coal = pd.merge(ar6_world_coal,ar6_scen_NDC[["Model","Scenario","Category"]],
                                  left_on=["Model","Scenario"],right_on=["Model","Scenario"])
    ar6_world_coal_melt = pd.melt(ar6_world_coal,id_vars=["Model","Scenario","Category","Region"],value_vars=ar6_world_coal.columns[5:-1],var_name="Year",value_name=value_name).dropna()
    ar6_world_coal_melt["Year"] = ar6_world_coal_melt["Year"].astype(int)
    ar6_world_coal_melt = ar6_world_coal_melt.sort_values("Category")
    ar6_world_coal_melt.set_index("Model").to_csv(file_name)
ar6_world_coal_melt

,Model,Scenario,Category,Region,Year,EJ/yr
21476,MESSAGEix-GLOBIOM_1.1,NGFS2_Nationally Determined Contributions (NDCs),C5,R10AFRICA,2048,1.111644
30210,REMIND 2.1,R2p1_SSP2-NDC,C5,R10CHINA+,2076,24.573780
30209,REMIND 2.1,R2p1_SSP2-NDC,C5,R10AFRICA,2076,24.240200
30208,REMIND 2.1,R2p1_SSP1-NDC,C5,R10ROWO,2076,2.866880
30207,REMIND 2.1,R2p1_SSP1-NDC,C5,R10REST_ASIA,2076,17.420220
...,...,...,...,...,...,...
13129,IMAGE 3.0,CO_NDCplus,C7,R10NORTH_AM,2016,0.877814
13128,IMAGE 3.0,CO_NDCplus,C7,R10LATIN_AM,2016,0.073706
13127,IMAGE 3.0,CO_NDCplus,C7,R10EUROPE,2016,0.045570
13136,IMAGE 3.0,EN_INDCi2100,C7,R10MIDDLE_EAST,2016,0.004070
